In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import random

/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/

In [2]:
df_test = pd.read_csv("test2.csv")

df_train0 = pd.read_csv("train.csv")
df_train1 = pd.read_csv("train1.csv")
df_train2 = pd.read_csv("train2.csv")
df_train3 = pd.read_csv("train3.csv")
df_train_list = [df_train0,df_train1, df_train2, df_train3]
df_train3.head()

,tollgate_id,time_window,direction,volume,time,date,hour,miniute,am_pm,0,1,2,3,4,5,period_num,precipitation,rel_humidity
0,1,2016-09-19 06:15:00,0,14,2016-09-19 06:15:00,2016-09-19,6,15,1,10,9,8,18,5,10,0.75,0.0,56.0
1,1,2016-09-19 06:35:00,0,30,2016-09-19 06:35:00,2016-09-19,6,35,1,10,9,8,18,5,10,1.75,0.0,56.0
2,1,2016-09-19 06:55:00,0,32,2016-09-19 06:55:00,2016-09-19,6,55,1,10,9,8,18,5,10,2.75,0.0,56.0
3,1,2016-09-19 07:15:00,0,34,2016-09-19 07:15:00,2016-09-19,7,15,1,10,9,8,18,5,10,0.75,NaN,NaN
4,1,2016-09-19 07:35:00,0,38,2016-09-19 07:35:00,2016-09-19,7,35,1,10,9,8,18,5,10,1.75,NaN,NaN


In [3]:
def feature_transform_split(key, data):
    # data = remove_exception(data)

    data["precipitation"] = data[["precipitation"]].fillna(value=0)
    data["rel_humidity"] = data[["rel_humidity"]].fillna(value=50)

    data["precipitation"] = data["precipitation"].apply(lambda x: x > 0)
    data["rel_humidity"] = data["rel_humidity"].apply(lambda x: x > 90)

    data = data.drop("precipitation", axis=1)
    # data = data.drop("rel_humidity", axis= 1)


    # data["sum"] = data["0"] + data["1"] + data["2"] + data["3"] + data["4"] + data["5"]

    data = pd.concat([data, pd.get_dummies(data['period_num'])], axis=1)
    data = data.drop("period_num", axis=1)

    data = pd.concat([data, pd.get_dummies(data['holiday'])], axis=1)
    data = data.drop("holiday", axis=1)
    #
    # data = pd.concat([data, pd.get_dummies(data['first_last_workday'])], axis=1)
    data = data.drop("first_last_workday", axis=1)

    data = data.drop("day_of_week", axis=1)

    if (key == 1):
        data = pd.concat([data, pd.get_dummies(data['tollgate_id'])], axis=1)
        # data["tollgate_id1"] = data['tollgate_id']
        data["direction1"] = data['direction']
    return data

In [4]:
#df_hue = pd.concat(df_train_list)
#df_hue.to_csv("result/result_split_rf_huehue"+"huehue"+".csv", index=False)
random.shuffle(df_train_list)
df_train = pd.concat(df_train_list)

#df_ts = pd.read_csv("ts_feature2_simple.csv")
df_date = pd.read_csv("date.csv")

df_train = df_train.merge(df_date, on="date", how="left")
#df_train = df_train.merge(df_ts, on=["tollgate_id", "hour", "miniute", "direction"], how="left")

df_test = df_test.merge(df_date, on="date", how="left")
#df_test = df_test.merge(df_ts, on=["tollgate_id", "hour", "miniute", "direction"], how="left")

df_train_grouped = df_train.groupby(["tollgate_id", "direction"])
df_test_grouped = df_test.groupby(["tollgate_id", "direction"])
result = []
oob = []
for key, train_data in df_train_grouped:
    test_data = df_test_grouped.get_group(key)
    len_train = len(train_data)
    train_data = train_data.append(test_data)[train_data.columns.tolist()]
    train_data = feature_transform_split(key, train_data)
    train_data.head()

    regressor_cubic = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=10, oob_score=True)

    train_data = pd.DataFrame.reset_index(train_data)
    train_data = train_data.drop("index", axis=1)
    y = train_data.ix[:len_train - 1, :]["volume"]

    
    x = train_data.ix[:len_train - 1, 8:]
    x1 = train_data.ix[len_train:, 8:]
    regressor_cubic.fit(x, y)
    print(x1)
    resultado_obtido = regressor_cubic.predict(x1)

    #df_h = test_data
    #df_h.to_csv("result/result_split_rf_huehue"+"teste"+".csv", index=False)

    test_data["volume"] = resultado_obtido
    result.append(test_data[['tollgate_id', 'time_window', 'direction', 'volume']])

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


      am_pm  0  1  2  3  4  5  rel_humidity  0.0  0.25 ...  1.25  1.5  1.75  \
1728      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1729      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1730      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1731      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1732      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1733      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1734      0  0  0  0  0  0  0          True    1     0 ...     0    0     0   
1735      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1736      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1737      0  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1738      0  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1739      0  0  0  0  0  0  0         False    0    

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib

      am_pm  0  1  2  3  4  5  rel_humidity  0.0  0.25 ...  1.25  1.5  1.75  \
1728      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1729      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1730      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1731      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1732      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1733      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1734      0  0  0  0  0  0  0          True    1     0 ...     0    0     0   
1735      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1736      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1737      0  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1738      0  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1739      0  0  0  0  0  0  0         False    0    

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib

...

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib

      am_pm  0  1  2  3  4  5  rel_humidity  0.0  0.25 ...  1.25  1.5  1.75  \
1728      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1729      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1730      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1731      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1732      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1733      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1734      0  0  0  0  0  0  0          True    1     0 ...     0    0     0   
1735      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1736      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1737      0  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1738      0  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1739      0  0  0  0  0  0  0         False    0    

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/matheuspds/miniconda3/lib

      am_pm  0  1  2  3  4  5  rel_humidity  0.0  0.25 ...  1.25  1.5  1.75  \
1728      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1729      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1730      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1731      1  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1732      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1733      1  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1734      0  0  0  0  0  0  0          True    1     0 ...     0    0     0   
1735      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1736      0  0  0  0  0  0  0          True    0     0 ...     0    0     0   
1737      0  0  0  0  0  0  0         False    1     0 ...     0    0     0   
1738      0  0  0  0  0  0  0         False    0     0 ...     0    0     0   
1739      0  0  0  0  0  0  0         False    0    

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df_result = pd.concat(result, axis=0)
df_result.to_csv("resultado_final"+"verificar"+".csv", index=False)
df_result.head()

,tollgate_id,time_window,direction,volume
0,1,2016-10-25 06:00:00,0,40.113246
1,1,2016-10-25 06:20:00,0,41.847567
2,1,2016-10-25 06:40:00,0,45.986700
3,1,2016-10-25 07:00:00,0,40.113246
4,1,2016-10-25 07:20:00,0,41.847567


In [117]:
df_ = pd.read_csv("test2.csv")
df_.count()

tollgate_id      420
time_window      420
direction        420
volume           420
time             420
date             420
hour             420
miniute          420
am_pm            420
0                420
1                420
2                420
3                420
4                420
5                420
period_num       420
precipitation    210
rel_humidity     210
dtype: int64

In [92]:
df_test_real = pd.read_csv("resultado_real_teste.csv")

#df_train0 = pd.read_csv("train.csv")
#df_train1 = pd.read_csv("train1.csv")

In [93]:
df_test_real.head()

,time,tollgate_id,direction,time_window,volume
0,2016-10-25 06:00:00,1,0,"[06:00:00,06:20:00)",10
1,2016-10-25 06:00:00,1,1,"[06:00:00,06:20:00)",37
2,2016-10-25 06:00:00,2,0,"[06:00:00,06:20:00)",29
3,2016-10-25 06:00:00,3,0,"[06:00:00,06:20:00)",28
4,2016-10-25 06:00:00,3,1,"[06:00:00,06:20:00)",30


In [94]:
df_result.head()

,tollgate_id,time_window,direction,volume
0,1,2016-10-25 08:00:00,0,56.229729
1,1,2016-10-25 08:20:00,0,57.821395
2,1,2016-10-25 08:40:00,0,55.578495
3,1,2016-10-25 09:00:00,0,54.183862
4,1,2016-10-25 09:20:00,0,56.229729


In [ ]:
def feature_format():
    v_train = pd.read_csv('dados_treino_volume_com_valor_anterior.csv')
    v_test = pd.read_csv('dados_teste_volume_com_valor_anterior.csv')
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(v_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    v_train['window_n'] = v_train['time_window'].map(s)
    v_test['window_n'] = v_test['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = v_train.drop('volume', axis = 1)
    feature_test = v_test.drop('volume',axis = 1)
    values_train = v_train['volume'].values
    values_test = v_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [95]:
values_test_real = df_test_real['volume'].values
values = df_result['volume'].values.astype(int)
values_test_prediction = values

In [97]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [98]:
mean_absolute_percentage_error(values_test_real, values_test_prediction)

119.32587923038267

In [96]:
values_test_prediction

array([ 56,  57,  55,  54,  56,  57,  51,  51,  49,  46,  47,  50,  56,
        57,  55,  54,  56,  57,  51,  51,  49,  46,  47,  50,  56,  57,
        55,  54,  56,  57,  51,  51,  49,  46,  47,  50,  56,  57,  55,
        54,  56,  57,  51,  51,  49,  46,  47,  50,  67,  69,  69,  69,
        67,  69,  62,  63,  63,  62,  62,  63,  67,  69,  69,  69,  67,
        69,  62,  63,  63,  62,  62,  63,  56,  57,  55,  54,  56,  57,
        51,  51,  49,  48,  51,  51,  61,  63,  61,  64,  61,  63,  58,
        59,  59,  57,  56,  57,  61,  63,  61,  64,  61,  63,  58,  59,
        59,  57,  56,  57,  61,  63,  61,  64,  61,  63,  58,  59,  59,
        57,  56,  57,  61,  63,  61,  64,  61,  63,  58,  59,  59,  57,
        56,  57,  49,  49,  48,  50,  49,  49,  47,  48,  47,  48,  47,
        48,  49,  49,  48,  50,  49,  49,  47,  48,  47,  48,  47,  48,
        61,  63,  61,  64,  61,  63,  58,  59,  59,  59,  58,  59, 100,
        98, 102,  99, 100,  98,  91,  89,  93,  85,  83,  76, 10

In [84]:
values_test_real

array([ 10,  37,  29,  28,  30,  17,  54,  26,  48,  31,  30,  60,  40,
        91,  41,  25,  66,  83, 124,  59,  38,  99,  95, 115,  84,  44,
       132,  89, 134,  81,  52,  91,  76, 117,  76,  42, 108,  64, 119,
       101,  46,  88,  98, 117,  79,  47, 110,  78, 122,  91,  40, 102,
        83, 137, 100,  50, 103,  68, 113,  96,  12,  37,  32,  24,  21,
         9,  50,  27,  50,  36,  25,  57,  39,  73,  41,  31,  68,  73,
       123,  50,  31,  94,  86, 116,  78,  45, 105,  94, 134,  85,  38,
        91,  71, 111,  77,  35,  93,  58, 101,  78,  44,  99,  73, 108,
        77,  49, 105,  67, 121,  80,  45, 108,  80, 129,  91,  45, 103,
        89, 121, 118,  14,  37,  20,  23,  24,  14,  46,  36,  40,  25,
        22,  51,  50,  61,  49,  20,  61,  68, 116,  51,  36,  92,  80,
       107,  89,  42,  99, 113, 126,  84,  39,  84,  58, 104,  96,  37,
        98,  53,  90,  87,  45,  95,  79,  98,  78,  49,  93,  57, 107,
       102,  41,  89,  72, 118, 106,  45, 101,  80, 117, 113,   

In [100]:
mape = np.mean(np.abs((values_test_prediction - values_test_real)/values_test_real))
mape

1.1932587923038267